# Brian Replication of data_model.py in a python notebook

In [166]:
import pandas as pd
import logging
import dateutil
from dateutil import parser
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 500)

In [141]:
logging.basicConfig(level=logging.INFO)

# Code Broken Down

In [142]:
df = pd.read_csv("cleaned/all_quarters_merged_TEST.csv")

/Users/briangoggin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1,2,6,12,16,17,20,21,27,28,29,30,31,32,34,35,36,37,38,39,40,41,42,43,44,45,46,47,49,50,51,59,77,86,89,90,91,92,94,95,96,97,98,99,101,111,112,113,114,115,116,117,118,119,120,121,138,140,141,142,143,144) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [144]:
# keep only variables of interest
list = ['address', 'apn', 'best_date', 'best_stat','firstfiled', 'report_quarter', 'report_year', 'units', 'unitsnet', 'dbi_permit', 'x', 'y']
df = df[list]

In [145]:
#consolidate status categories. Start with 3 for now
def status_function(value):
    if value['best_stat']=="CONSTRUCTION":
        field = 'Under Construction'
    elif (value['best_stat']=='BP APPROVED') | (value['best_stat']=='BP ISSUED') | (value['best_stat']=='BP REINSTATED'):
        field = 'Building Permit Approved'
    else:
        field = 'Proposed'
    return field
        
df['status']=df.apply(status_function, axis=1)

In [146]:
gb = df.groupby(['apn', 'address'])

In [147]:
gb.groups

{('3594023',
  '835 SOUTH VAN NESS AV'): Int64Index([9385, 9919, 11067, 12456, 13364, 14826], dtype='int64'),
 ('4045002',
  '740 ILLINOIS ST AND 2121 THIRD ST'): Int64Index([328, 983, 2222, 2739, 3468, 4263, 4941, 5850, 6704, 7578], dtype='int64'),
 ('6250037',
  '95 LELAND AV'): Int64Index([5289, 6249, 6950, 7902, 8575, 10573, 11229, 12395], dtype='int64'),
 ('2636001', '1530 - 1585  5TH AV'): Int64Index([13271, 14851], dtype='int64'),
 ('3577053',
  '33 DEARBORN ST'): Int64Index([11298, 12450, 13677, 15264], dtype='int64'),
 ('6655030',
  '176 RANDALL ST'): Int64Index([  256,  1204,  2198,  2617,  3376,  3980,  4846,  5763,  6467,
              7487,  8997,  9634, 10545, 11500, 12770, 13961, 14935],
            dtype='int64'),
 ('2613020',
  '2317 15TH ST'): Int64Index([  340,   919,  1992,  2770,  3496,  4047,  4868,  5780,  6731,
              7594,  8436,  9691, 10585, 11537, 12682, 14030, 15083],
            dtype='int64'),
 ('1612017N',
  '774 33RD AV'): Int64Index([ 3698,  413

In [148]:
group_df = gb.get_group(('0829040','526 HICKORY ST'))

In [149]:
group_df = group_df.sort('best_date')
group_df.reset_index(drop=True, inplace=True)

/Users/briangoggin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [150]:
group_df

,address,apn,best_date,best_stat,firstfiled,report_quarter,report_year,units,unitsnet,dbi_permit,x,y,status
0,526 HICKORY ST,0829040,2015-01-21,CONSTRUCTION,NaN,2,2015,1.0,1.0,201209200264,37.774845,-122.428663,Under Construction


In [97]:
# identify building permit ID. Then fill in rest of quarters with this permit ID.
building_permit=np.nan
for index, row in group_df.iterrows():
    if pd.isnull(row['dbi_permit']) and not pd.isnull(building_permit):
        building_permit=building_permit
    else:
        building_permit = row['dbi_permit']

In [98]:
building_permit

'201101208700'

In [99]:
group_df['dbi_permit']=building_permit

In [54]:
#Identify completion quarter
for index, row in group_df.iterrows():
    if row['best_stat'] !='CONSTRUCTION':
        comp_quarter= np.nan
        comp_year=np.nan
    elif row['best_stat'] == 'CONSTRUCTION':
        comp_quarter = row['report_quarter']
        comp_year = row['report_year']

if pd.isnull(comp_quarter):
    pass
if comp_quarter == 4 and comp_year == 2016:
    comp_quarter = np.nan
elif comp_quarter == 4:
    comp_quarter = 1
    comp_year = 1+comp_year
else:
    comp_quarter = 1+comp_quarter
    

In [59]:
if comp_quarter ==1:
    comp_daymth= '01/01'
elif comp_quarter ==2:
    comp_daymth= '04/01'
elif comp_quarter==3:
    comp_daymth= '07/01'
elif comp_quarter == 4:
    comp_daymth= '10/01'
elif pd.isnull(comp_quarter):
    comp_daymth=np.nan
    
if pd.isnull(comp_daymth):
    comp_date = np.nan
else:
    comp_date = comp_daymth + "/" + str(comp_year)

In [60]:
comp_date

nan

In [ ]:
#Identify earliest "firstfiled" date
firstfiled = ''
for index, row in group_df.iterrows():
    if pd.isnull(row['firstfiled']):
        continue
    else:
        if len(firstfiled) == 0:
            firstfiled = row['firstfiled']
        else:
            continue

In [ ]:
#Identify earliest best date
earliest_BD = ''
for index, row in group_df.iterrows():
    if pd.isnull(row['best_date']):
        continue
    else:
        if len(earliest_BD) == 0:
            earliest_BD = row['best_date']
        else:
            continue

In [ ]:
firstfiled

In [ ]:
earliest_BD

In [ ]:
# Finalize first date variable (minimum of earliest best_date and firstfiled)
if firstfiled =='' and earliest_BD !='':
    first_date = earliest_BD
elif earliest_BD=='' and firstfiled !='':
    first_date = firstfiled
elif firstfiled !='' and earliest_BD !='':
    first_date = min(firstfiled, earliest_BD)

In [ ]:
first_date

In [ ]:
# initiate variables. Groups without these dates are blank for these variables.
BP_date = ''
con_date = ''

#Identify first date for all status categories
m=0
for index, row in group_df.iterrows():
    if m == 0:
        status_previous = 'blah'
    if row['status']=='Building Permit Approved':
        if index == 0:
            BP_date = row['best_date']
        elif index !=0:
            if status_previous =='Building Permit Approved':
                BP_date=BP_date
            else:
                BP_date = row['best_date']
    elif row['status']=='Under Construction':
        if index == 0:
            con_date = row['best_date']
        elif index !=0:
            if status_previous =='Under Construction':
                con_date=con_date
            else:
                con_date = row['best_date']
    status_previous = row['status']
    m=m+1

In [ ]:
print (BP_date, con_date)

In [ ]:
last_row = group_df.tail(1).copy() #most recent row

In [ ]:
last_row

In [ ]:
last_row['latest_project_record_date'] = last_row.best_date
last_row['first_project_record_date'] = group_df.iloc[0].best_date
last_row['latest_project_status'] = last_row.best_stat

In [ ]:
last_row

In [ ]:
## Store a parseable list of all the project states and the dates those states were reported
last_row['project_dates'] = str(tuple(group_df.best_date))
last_row['project_statuses'] = str(tuple(group_df.best_stat))

In [ ]:
last_row

In [ ]:
## Store the project duration in days
latest = last_row.latest_project_record_date.iloc[0]
first = last_row.first_project_record_date.iloc[0]


In [ ]:
if not (pd.isnull(latest) or pd.isnull(first)):
    last_row['project_duration_days'] = (dateutil.parser.parse(latest) - dateutil.parser.parse(first)).days

In [ ]:
last_row

# Actual Code

In [169]:
def convert_to_one_record_per_project(df):
    """
    Group the dataset by (apn, address) and then emit one row per (apn, address) pair.
    best_date and best_stat will be converted into arrays.
    All other attributes will be taken from the record with the most recent `best_date` attribute.
    """
    gb = df.groupby(['apn', 'address'])
    for k in gb.groups:
        group_df = gb.get_group(k)
        group_df = group_df.sort('best_date')
        last_row = group_df.tail(1).copy()
        
        # identify building permit ID. Then fill in rest of quarters with this permit ID.
        building_permit=np.nan
        for index, row in group_df.iterrows():
            if pd.isnull(row['dbi_permit']) and not pd.isnull(building_permit):
                building_permit=building_permit
            else:
                building_permit = row['dbi_permit']
                
        #Identify completion quarter for those projects that have reached completion
        for index, row in group_df.iterrows():
            if row['best_stat'] !='CONSTRUCTION':
                comp_quarter= np.nan
                comp_year=np.nan
            elif row['best_stat'] == 'CONSTRUCTION':
                comp_quarter = row['report_quarter']
                comp_year = row['report_year']

        if pd.isnull(comp_quarter):
            pass
        if comp_quarter == 4 and comp_year == 2016:
            comp_quarter = np.nan
        elif comp_quarter == 4:
            comp_quarter = 1
            comp_year = 1+comp_year
        else:
            comp_quarter = 1+comp_quarter
            
        if comp_quarter ==1:
            comp_daymth= '01/01'
        elif comp_quarter ==2:
            comp_daymth= '04/01'
        elif comp_quarter==3:
            comp_daymth= '07/01'
        elif comp_quarter == 4:
            comp_daymth= '10/01'
        elif pd.isnull(comp_quarter):
            comp_daymth=np.nan

        if pd.isnull(comp_daymth):
            comp_date = np.nan
        else:
            comp_date = comp_daymth + "/" + str(comp_year)
        
        #Identify earliest "firstfiled" date
        firstfiled = ''
        for index, row in group_df.iterrows():
            if pd.isnull(row['firstfiled']):
                continue
            else:
                if len(firstfiled) == 0:
                    firstfiled = row['firstfiled']
                else:
                    continue
        
        #Identify earliest best date
        earliest_BD = ''
        for index, row in group_df.iterrows():
            if pd.isnull(row['best_date']):
                continue
            else:
                if len(earliest_BD) == 0:
                    earliest_BD = row['best_date']
                else:
                    continue
        
        # Finalize first date variable (minimum of earliest best_date and firstfiled)
        if firstfiled =='' and earliest_BD !='':
            first_date = earliest_BD
        elif earliest_BD=='' and firstfiled !='':
            first_date = firstfiled
        elif firstfiled !='' and earliest_BD !='':
            first_date = min(firstfiled, earliest_BD)
            
        # initiate variables. Groups without these dates are blank for these variables.
        BP_date = ''
        con_date = ''

        #Identify first date for all status categories
        m=0
        for index, row in group_df.iterrows():
            if m == 0:
                status_previous = 'blah'
            if row['status']=='Building Permit Approved':
                if index == 0:
                    BP_date = row['best_date']
                elif index !=0:
                    if status_previous =='Building Permit Approved':
                        BP_date=BP_date
                    else:
                        BP_date = row['best_date']
            elif row['status']=='Under Construction':
                if index == 0:
                    con_date = row['best_date']
                elif index !=0:
                    if status_previous =='Under Construction':
                        con_date=con_date
                    else:
                        con_date = row['best_date']
            status_previous = row['status']
            m=m+1
        
        last_row['dbi_permit']= building_permit
        last_row['comp_date']=comp_date
        last_row['BP_date'] = BP_date
        last_row['con_date'] = con_date
        last_row['first_date']=first_date
        last_row['latest_project_record_date'] = last_row.best_date
        last_row['first_project_record_date'] = group_df.iloc[0].best_date
        last_row['latest_project_status'] = last_row.best_stat

        ## Store a parseable list of all the project states and the dates those states were reported
        last_row['project_dates'] = str(tuple(group_df.best_date))
        last_row['project_statuses'] = str(tuple(group_df.best_stat))

        ## Store the project duration in days
        if not (pd.isnull(comp_date) or pd.isnull(first_date)):
            last_row['project_duration_days'] = (dateutil.parser.parse(comp_date) - dateutil.parser.parse(first_date)).days

        yield last_row


def main():
    df = pd.read_csv("cleaned/all_quarters_merged_TEST.csv")
    #consolidate status categories. Start with 3 for now
    def status_function(value):
        if value['best_stat']=="CONSTRUCTION":
            field = 'Under Construction'
        elif (value['best_stat']=='BP APPROVED') | (value['best_stat']=='BP ISSUED') | (value['best_stat']=='BP REINSTATED'):
            field = 'Building Permit Approved'
        else:
            field = 'Proposed'
        return field

    df['status']=df.apply(status_function, axis=1)
    
    #keep only variables we want
    list = ['address', 'apn', 'best_date', 'best_stat','status', 'firstfiled', 'report_quarter', 'report_year','units','unitsnet', 'dbi_permit', 'x', 'y']
    df = df[list]
    new_df = pd.concat(convert_to_one_record_per_project(df))
    logging.info("Writing output ({} rows, {} cols) to data/cleaned/all_quarters__one_record_per_project.csv".format(*new_df.shape))
    new_df.to_csv("cleaned/all_quarters__one_record_per_project_BRIAN.csv")
    return new_df

In [170]:
new_df = main()

/Users/briangoggin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (1,2,6,12,16,17,20,21,27,28,29,30,31,32,34,35,36,37,38,39,40,41,42,43,44,45,46,47,49,50,51,59,77,86,89,90,91,92,94,95,96,97,98,99,101,111,112,113,114,115,116,117,118,119,120,121,138,140,141,142,143,144) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/briangoggin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [171]:
new_df.head()

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y
14826,,835 SOUTH VAN NESS AV,3594023,2014-12-11,BP FILED,NaN,,2.01412e+11,2014-12-11,2014-12-11,2014-12-11,2014-12-11,BP FILED,"('2014-12-11', '2014-12-11', '2014-12-11', '20...",NaN,"('BP FILED', 'BP FILED', 'BP FILED', 'BP FILED...",1,2016,Proposed,3.0,3.0,37.759804,-122.416615
7578,2012-07-20,740 ILLINOIS ST AND 2121 THIRD ST,4045002,2014-05-23,CONSTRUCTION,07/01/2014,2012-11-13,201111179162,2010-02-12 08:00:00,2011-11-17,2010-02-12,2014-05-23,CONSTRUCTION,"('2011-11-17', '2012-07-20', '2012-07-20', '20...",1599.0,"('BP FILED', 'BP ISSUED', 'BP ISSUED', 'CONSTR...",2,2014,Under Construction,106.0,106.0,37.762685,-122.388248
12395,2010-03-05,95 LELAND AV,6250037,2010-03-05,BP REINSTATED,NaN,,2.00704e+11,2007-04-12,2010-03-05,2007-04-12,2010-03-05,BP REINSTATED,"('2010-03-05', '2010-03-05', '2010-03-05', '20...",NaN,"('BP REINSTATED', 'BP REINSTATED', 'BP REINSTA...",3,2015,Building Permit Approved,8.0,5.0,37.711721,-122.405714
14851,,1530 - 1585 5TH AV,2636001,2015-10-22,PL FILED,NaN,,NaN,2014-10-15,2015-10-22,2014-10-15,2015-10-22,PL FILED,"('2015-10-22', '2015-10-22')",NaN,"('PL FILED', 'PL FILED')",1,2016,Proposed,460.0,374.0,37.759368,-122.460648
15264,,33 DEARBORN ST,3577053,2015-07-10,BP FILED,NaN,,2.01507e+11,2015-06-10,2015-07-10,2015-06-10,2015-07-10,BP FILED,"('2015-07-10', '2015-07-10', '2015-07-10', '20...",NaN,"('BP FILED', 'BP FILED', 'BP FILED', 'BP FILED')",1,2016,Proposed,7.0,3.0,37.762699,-122.422774


In [172]:
#First, filter out those projects that are exclusively non-residential (defined as those without units)
new_df = new_df[new_df['units'] > 0]

In [173]:
#Next, keep only those that reached completion at some point over the time period
new_df=new_df[pd.notnull(new_df['comp_date'])]

In [174]:
new_df.head()

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y
7578,2012-07-20,740 ILLINOIS ST AND 2121 THIRD ST,4045002,2014-05-23,CONSTRUCTION,07/01/2014,2012-11-13,201111179162,2010-02-12 08:00:00,2011-11-17,2010-02-12,2014-05-23,CONSTRUCTION,"('2011-11-17', '2012-07-20', '2012-07-20', '20...",1599.0,"('BP FILED', 'BP ISSUED', 'BP ISSUED', 'CONSTR...",2,2014,Under Construction,106.0,106.0,37.762685,-122.388248
15465,2013-07-22,497 VALLEY ST,6621030,2015-08-26,CONSTRUCTION,04/01/2016,2015-02-03,2.00701e+11,2007-01-10,2007-01-10,2007-01-10,2015-08-26,CONSTRUCTION,"('2007-01-10', '2007-01-10', '2007-01-10', '20...",3369.0,"('BP FILED', 'BP FILED', 'BP FILED', 'BP FILED...",1,2016,Under Construction,1.0,1.0,37.743992,-122.433053
6933,2013-12-18,263 LEXINGTON ST,3596059,2014-03-20,CONSTRUCTION,04/01/2014,2014-03-20,201305287999,2013-05-28,2013-05-28,2013-05-28,2014-03-20,CONSTRUCTION,"('2013-05-28', '2013-05-28', '2013-12-18', '20...",308.0,"('BP FILED', 'BP FILED', 'BP ISSUED', 'CONSTRU...",1,2014,Under Construction,2.0,1.0,37.759194,-122.420376
442,,209 09TH ST,3729081,2012-02-07,CONSTRUCTION,04/01/2012,2012-02-07,200810164336,2005-10-13 07:00:00,2012-02-07,2005-10-13 07:00:00,2012-02-07,CONSTRUCTION,"('2012-02-07',)",2361.0,"('CONSTRUCTION',)",1,2012,Under Construction,3.0,3.0,37.774940,-122.412852
13380,,6524 FULTON ST,1684002I,2011-06-03,CONSTRUCTION,01/01/2016,2011-06-03,2.00712e+11,2007-12-20 08:00:00,2011-06-03,2007-12-20,2011-06-03,CONSTRUCTION,"('2011-06-03', '2011-06-03', '2011-06-03', '20...",2933.0,"('CONSTRUCTION', 'CONSTRUCTION', 'CONSTRUCTION...",4,2015,Under Construction,2.0,1.0,37.772017,-122.501834


In [175]:
#Finally, filter out records with duplicate BP Ids
new_df[new_df.duplicated('dbi_permit', keep=False)]['address'].unique()


array(['1190 MISSION ST', '1420 MISSION ST', '57 TEHAMA ST',
       '1169 MARKET ST', '1000 16TH ST', '41 TEHAMA ST',
       '55 LAGUNA STREET', '1400 MISSION ST',
       '5 SHIPLEY ST / 935 FOLSOM ST / 77 FALMOUTH ST',
       '1076 HAMPSHIRE ST', '690 MONTEREY BLVD', '4132 03RD ST',
       '800 BROTHERHOOD', '1446 OCEAN AV', '8 OCTAVIA ST', '1239 TURK ST',
       '344 - 388 FULTON ST', '1078 HAMPSHIRE ST', '800 BROTHERHOOD WAY',
       '218 BUCHANAN ST', 'HP SHIPYARD PHASE 1 (UNDER CONSTRUCTION)',
       '650 INDIANA ST', '55 LAGUNA ST (BLDG 2)', '1400 07TH ST',
       '4132 THIRD STREET', '2155 WEBSTER ST',
       '236 - 238 SHIPLEY ST / 77 FALMOUTH ST', '447 - 453 LINDEN ST',
       '233 - 237 SHIPLEY ST', '650 INDIANA ST (BLDG 2 OF 2)',
       '1100 GOLDEN GATE AV', '447 LINDEN ST', '4 OCTAVIA ST',
       'HUNTERS POINT SHIPYARD PHASE I', '2101 & 2155 WEBSTER STREET',
       '55 LAGUNA ST', '690 MONTEREY BL', '1490 OCEAN AV',
       '233-237 SHIPLEY ST', 'PARCEL F'], dtype=object)

In [158]:
#explore duplicated
new_df[new_df['address']=='1190 MISSION ST']

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y
4975,,1190 MISSION ST,3702052,2013-05-15,CONSTRUCTION,10/01/2013,2012-03-08,201012166843,2010-12-16 08:00:00,2012-03-08,2010-12-16,2013-05-15,CONSTRUCTION,"('2012-03-08', '2012-07-11', '2012-10-09', '20...",433.0,"('CONSTRUCTION', 'CONSTRUCTION', 'CONSTRUCTION...",3,2013,Under Construction,418.0,418.0,37.77762,-122.412985


In [159]:
# Same project, different address-apn combination—verified by looking at each address in PIM
new_df[new_df['dbi_permit']=='201012166843']

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y
4975,,1190 MISSION ST,3702052,2013-05-15,CONSTRUCTION,10/01/2013,2012-03-08,201012166843,2010-12-16 08:00:00,2012-03-08,2010-12-16,2013-05-15,CONSTRUCTION,"('2012-03-08', '2012-07-11', '2012-10-09', '20...",433.0,"('CONSTRUCTION', 'CONSTRUCTION', 'CONSTRUCTION...",3,2013,Under Construction,418.0,418.0,37.777620,-122.412985
425,,1169 MARKET ST,3702053,2012-03-08,CONSTRUCTION,04/01/2012,2012-03-08,201012166843,2003-07-15 07:00:00,2012-03-08,2003-07-15 07:00:00,2012-03-08,CONSTRUCTION,"('2012-03-08',)",0.0,"('CONSTRUCTION',)",1,2012,Under Construction,418.0,418.0,37.779181,-122.414016


In [164]:
#explore duplicated
new_df[new_df['address']=='2155 WEBSTER ST']

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y
9106,,2155 WEBSTER ST,0629037,2015-03-18,CONSTRUCTION,01/01/2015,2015-03-18,201311222660,2013-11-22,2013-11-22,NaN,2015-03-18,CONSTRUCTION,"('2013-11-22', '2013-11-22', '2013-11-22', '20...",481.0,"('BP FILED', 'BP FILED', 'BP FILED', 'BP FILED...",4,2014,Under Construction,77.0,77.0,37.79057,-122.432337


In [165]:
new_df[new_df['dbi_permit']=='201311222660']

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y
9106,,2155 WEBSTER ST,0629037,2015-03-18,CONSTRUCTION,01/01/2015,2015-03-18,201311222660,2013-11-22,2013-11-22,NaN,2015-03-18,CONSTRUCTION,"('2013-11-22', '2013-11-22', '2013-11-22', '20...",481.0,"('BP FILED', 'BP FILED', 'BP FILED', 'BP FILED...",4,2014,Under Construction,77.0,77.0,37.790570,-122.432337
12206,,2101 & 2155 WEBSTER STREET,0629037,2015-10-30,CONSTRUCTION,10/01/2015,2015-06-08,201311222660,2013-11-22,2015-06-08,2013-11-22,2015-10-30,CONSTRUCTION,"('2015-06-08', '2015-07-27', '2015-10-30')",144.0,"('CONSTRUCTION', 'CONSTRUCTION', 'CONSTRUCTION')",3,2015,Under Construction,76.0,76.0,37.790296,-122.432772


# Analysis Section

In [176]:
#convert completion date to date variables type
new_df['project_duration_days'].describe()

count     613.000000
mean     1751.290375
std      1194.313905
min       -20.000000
25%       827.000000
50%      1402.000000
75%      2518.000000
max      8806.000000
Name: project_duration_days, dtype: float64